## NLP As A Supervised Problem
--------------------------------------------------------
Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from Alice in Wonderland by Lewis Carroll or Persuasion by Jane Austen, using different supervised learning algorithms.

Our feature-generation approach will be something called BoW, or Bag of Words. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.

In [1]:
# Import libraries:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import gutenberg, stopwords
from collections import Counter

#nltk.download('gutenberg')
#!python -m spacy download en

In [2]:
# Create utility function for standard text cleaning:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [3]:
# Load Pursuation and Alice in wonderland:
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# Clean out the Chapter text as we won't need this for our model:
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
  
# Clean the datasets using text_cleaner function above:
# We will only use the first tenth of each text to avoid long execution time.
alice = text_cleaner(alice[:int(len(alice)/10)]) 
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

# Print out the first 1000 letter to inspect Alice in wonderland:
alice[:1000]

"Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?' So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, 'Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT-POCKET, and looked at it, and th

In [4]:
# Parse the cleaned novels:
nlp = spacy.load('en')

alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

alice_doc[:250]

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?' So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so VERY remarkable in that; nor did Alice think it so VERY much out of the way to hear the Rabbit say to itself, 'Oh dear! Oh dear! I shall be late!' (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually TOOK A WATCH OUT OF ITS WAISTCOAT-POCKET, and looked at it, and the

In [5]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words! Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur. We will exclude stopwords and punctuation. In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [6]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop] 
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]


# Creates a data frame with features for each word in our common word set. 
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation, stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 50 == 0:
            print("Processing row {}".format(i))
            
    return df

In [7]:
# Set up the BOWs for Alice in wornderlands and Persuasion:
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [8]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 50
Processing row 100
Processing row 150
Processing row 200
Processing row 250
Processing row 300
Processing row 350
Processing row 400
Processing row 450


,distinction,principally,happy,danger,contrary,Sir,difference,avert,dry,sense,...,book,glove,kid,alter,Stevenson,prudent,information,wind,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


### Building classification algorithms:

In [9]:
# Split the dataset into train & test set:
from sklearn.model_selection import train_test_split
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

#### 1. Author classification with Logistic regression:

In [10]:
# BOW with logistic regression:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty='l2') # No need to specify l2 as it's the default. But we put it for demonstration.
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(285, 1608) (285,)
Training set score: 0.968421052631579

Test set score: 0.8631578947368421


We definitely have overfitting issue with logistic regression. Let's try support vector machine and see if it will give us a better result

#### 2. Author classification with Support Vector Machine

In [11]:
# BOW with SVM:
from sklearn.svm import SVC

svc = SVC(kernel='sigmoid', random_state=123)
train = svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

(285, 1608) (285,)
Training set score: 0.8245614035087719

Test set score: 0.8105263157894737


Support vector machine seems to be better at not overfit our training set. However, our test result performed not as good as logistic regression. Let's try adding puctuations to the original dataset and see if result will improve

### Add punctuations to improve the performance

In [12]:
# Add punctuation
# Utility function to create a list of the 2000 most common words.
def words_punct(text):
    
    # Filter out stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_stop
                ]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords_punct = words_punct(alice_doc)
persuasionwords_punct = words_punct(persuasion_doc)

# Combine bags to create a set of unique words.
common_words_punct = set(alicewords_punct + persuasionwords_punct)

# Create data frame with features. This can take a while
word_counts_punct = bow_features(sentences, common_words_punct)
word_counts_punct.head()

Processing row 0


,distinction,principally,happy,danger,contrary,Sir,difference,avert,dry,sense,...,book,glove,kid,alter,Stevenson,prudent,information,wind,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [13]:
# Test new features using logistic regression:

Y = word_counts_punct['text_source']
X = word_counts_punct.drop(['text_sentence', 'text_source'], 1)

# Split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(285, 1621) (285,)
Training set score: 0.9859649122807017

Test set score: 0.9


That looks a lot better. Our test result came out to have 90% accuracy. Now let's try the model on another novel to see if our algorithm detects any problem.

### Try another novel:

In [14]:
# Check for available novels:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [15]:
# Grab Whitman, clean and process
whitman = gutenberg.raw('whitman-leaves.txt')
whitman = re.sub(r'VOLUME \w+', '', whitman)
whitman = re.sub(r'CHAPTER \w+', '', whitman)
whitman = text_cleaner(whitman[:int(len(whitman)/10)])
print(whitman[:500])

Come, said my soul, Such verses for my Body let us write, (for we are one,) That should I after return, Or, long, long hence, in other spheres, There to some group of mates the chants resuming, (Tallying Earth's soil, trees, winds, tumultuous waves,) Ever with pleas'd smile I may keep on, Ever and ever yet the verses owning as, first, I here and now Signing for Soul and Body, set to them my name, Walt Whitman } One's-Self I Sing One's-self I sing, a simple separate person, Yet utter the word Dem


In [16]:
# Parse into doc using SpaCy
whitman_doc = nlp(whitman[0:25000])

In [17]:
# Bag words
whitmanwords_punct = words_punct(whitman_doc)
alicewords_punct = words_punct(alice_doc)

# Combine Whitman and Alice bags
common_words = set(whitmanwords_punct + alicewords_punct)

# Group into sentences
whitman_sents = [[sent, 'Whitman'] for sent in whitman_doc.sents]

# Combine Whitman and Alice sentences
sentences = pd.DataFrame(whitman_sents + alice_sents)

# Get BoW Features
word_counts = bow_features(sentences, common_words)

Processing row 0


In [18]:
# Create target variable and explanatory variables
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)

In [20]:
# Combine Whitman and Persuasion bags
common_words = set(whitmanwords_punct + persuasionwords_punct)

# Combine Whitman and Persuasion sentences
sentences = pd.DataFrame(whitman_sents + persuasion_sents)

# Get BoW Features
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500


In [19]:
# Logistic regression with cross validataion:
from sklearn.model_selection import cross_val_score
print('Scores:', cross_val_score(lr, X, Y, cv=5))
print('Avg:', np.mean(cross_val_score(lr, X, Y, cv=5)))

Scores: [0.83950617 0.83950617 0.97530864 0.96296296 0.81481481]
Avg: 0.8864197530864196
